## Analyze Accuracy per class

In [ ]:
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import warnings

warnings.filterwarnings("ignore")

### topex-printer (RGB)

In [ ]:

ds_source = "../data/topex-printer/train"
ds_target = "../data/topex-printer/test"
df_classacc = pd.read_csv("../data/eval-data/STL-topex--uda--swinv2--ch-uda_cdan-mcc_adamw1e-5_warmupcalr_augmix_classacc.csv")
df_confmat = pd.read_csv("../data/eval-data/STL-topex--uda--swinv2--ch-uda_cdan-mcc_adamw1e-5_warmupcalr_augmix_confmat.csv")



In [ ]:
display(df_classacc.head())
display(df_classacc.describe())
display(df_confmat.head())

In [ ]:
def count_files(dir):
    return len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name))])

In [ ]:
IMSIZE = 150
IMSIZE2 = int(IMSIZE/2)
SPACER = 10
TEXTSIZE = 20

df_sorted = df_classacc.sort_values(by="acc", ascending=False)
for cls in df_sorted["class_name"]:
    img_source = Image.open(f"{ds_source}/{cls}/train_{cls}_0001_0010.png").resize((IMSIZE, IMSIZE))
    img_target = Image.open(f"{ds_target}/{cls}/test_{cls}_0001.png").resize((IMSIZE, IMSIZE))
    img_combi = Image.new("RGBA", (img_source.width + img_target.width + 10, img_source.height))
    img_combi.paste(img_source, (0, 0))
    img_combi.paste(img_target, (img_source.width + 10, 0))
    # display(img_combi)
    # ----------------------------------
    n_source = count_files(f"{ds_source}/{cls}")
    n_target = count_files(f"{ds_target}/{cls}")

    cls_conf = df_confmat[(df_confmat["Actual"] == cls) & (df_confmat["nPredictions"] > 0)]
    img_conf_source_combi = Image.new("RGBA", ((IMSIZE2 + SPACER)*len(cls_conf), IMSIZE2 + TEXTSIZE))
    img_conf_target_combi = Image.new("RGBA", ((IMSIZE2 + SPACER)*len(cls_conf), IMSIZE2 + TEXTSIZE))
    for i, pred_cls in enumerate(cls_conf["Predicted"]):
        ratio = cls_conf[(cls_conf["Predicted"] == pred_cls)]["nPredictions"] / n_target
        img_conf_source = Image.open(f"{ds_source}/{pred_cls}/train_{pred_cls}_0001_0010.png").resize((IMSIZE2, IMSIZE2))
        img_conf_source_combi.paste(img_conf_source, (i*(IMSIZE2+SPACER), 0))
        img_conf_target = Image.open(f"{ds_target}/{pred_cls}/test_{pred_cls}_0001.png").resize((IMSIZE2, IMSIZE2))
        img_conf_target_combi.paste(img_conf_target, (i*(IMSIZE2+SPACER), 0))

        font = ImageFont.load_default()
        text = f'{ratio.values[0]:.2f}'
        if pred_cls == cls:
            text_color = (255,255,255)
        else:
            text_color = (int(255*ratio.iloc[0]), 255-int(255*ratio.iloc[0]), 255-int(255*ratio.iloc[0]))
        draw = ImageDraw.Draw(img_conf_source_combi)
        text_width, text_height = draw.textsize(text, font=font)
        text_position = (i*(IMSIZE2+SPACER), IMSIZE2)  # Center the text
        draw.text(text_position, text, font=font, fill=text_color)

    display(img_conf_source_combi)
    display(img_conf_target_combi)

    print(f'ACC: {df_sorted[df_sorted["class_name"] == cls]["acc"].values[0]}')
    print(f"NAME: {cls}")
    print(f'N_SOURCE: {n_source}')
    print(f'N_TARGET: {n_target}')

